# Prioritized Experience Replay

> Fill in a module description here

In [ ]:
#| default_exp replay

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()

In [ ]:
#| export
import torch
from torch import nn
import torch.nn.functional as F
from torch.distributions import Bernoulli, Uniform

import numpy as np
from torchtyping import TensorType

### Prioritized Level Replay

In [ ]:
#| export
class PrioritizedReplayDistribution:
    def __init__(
        self,
        rho: float = 0.1,
        beta: float = 0.1, # the beta coefficient for the P_S distribution
    ) -> None:
        self.rho = rho
        self.beta = beta
    
    def create(
        self,
        score_levels: TensorType["batch_size", "n_seen_levels"],
        last_count_levels: TensorType["batch_size", "n_seen_levels"], # the last episode that the level was played,
        last_episode: TensorType["batch_size"] # the last episode of each batch
    ) -> TensorType["batch_size", "n_seen_levels"]:
        """Create a prioritized level distribution."""
        
        # scoring level distribution
        unnormalized_scores = torch.pow(
            input=F.normalize(score_levels, dim=-1),
            exponent=1/self.beta
        )
        score_dist = unnormalized_scores / unnormalized_scores.sum(dim=-1)[:, None]
        
        # staleness-awareness distribution
        stale_scores = last_episode - last_count_levels
        stale_dist = stale_scores / stale_scores.sum(dim=-1)
        
        prioritized_dist = (1 - self.rho) * score_dist + self.rho * stale_dist
        
        return prioritized_dist